In [10]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [11]:
import labeling_network as lbln
from matplotlib import pyplot as plt
import theano
import theano.tensor as T
import numpy as np
from labeling_network import FullyConnectedLayer, ConvPoolLayer

Running with a CPU.  If this is not desired, then the modify network3.py to set
the GPU flag to True.


In [12]:
import pylab
pylab.rcParams['figure.figsize'] = (6.0, 6.0)

In [ ]:
dataPath = '../../TrainingData/LabeledData-2015-09-07/'

In [ ]:
train_data, valid_data, test_data = lbln.load_data_shared(
                     dataPath,
                     'capture',
                     dataPath + 'labels.dat',
                     n_train=5000,
                     n_validation=500,
                     n_test=500)

# Flipping targets and inputs
train_data = (train_data[1], train_data[0])
valid_data = (valid_data[1], valid_data[0])
test_data = (test_data[1], test_data[0])

In [ ]:
mini_batch_size = 10

### Train the Network. 

In [ ]:
network = lbln.Network([
#         FullyConnectedLayer(n_in=29*29*20, n_out=128, p_dropout=0.5),
        FullyConnectedLayer(n_in=5*5, n_out=250),
        FullyConnectedLayer(n_in=250, n_out=64*64*3, p_dropout=0.0),        
    ], mini_batch_size)

In [ ]:
network.SGD(train_data, 
            1, 
            mini_batch_size, 
            1000.0, 
            valid_data, 
            test_data
           )

#### Save network weights.

In [ ]:
network.save_as_file('encoder_0_0108')

In [ ]:
network = network.load_from_file('filename', 10)

In [ ]:
train_data_xs = train_data[0].get_value()
train_data_ts = train_data[1].get_value()

test_data_xs = test_data[0].get_value()
test_data_ts = test_data[1].get_value()

valid_data_xs = valid_data[0].get_value()
valid_data_ts = valid_data[1].get_value()


In [ ]:
def show_image(flat_array, width=64, height=64, colors=3):
    plt.imshow(flat_array.reshape(width, height, colors), interpolation='nearest')
    plt.show()


In [ ]:
for index in xrange(10):
    show_image(test_data_ts[index])
    show_image(network.get_single_output(test_data_xs[index]))
    print test_data_xs[index]
    print network.get_single_output(test_data_xs[index])


### Determine all Losses on test images.

In [ ]:
def print_classification(index):
    print 'Target:    ', classifications[index][1]
    print 'Prediction:', classifications[index][2]
    print 'Loss:      ', classifications[index][3]
    show_image(classifications[index][1])
    show_image(classifications[index][2])


# Determine all errors on the test images
classifications = []
for x, t in zip(test_data_xs, test_data_ts):
    prediction = network.get_single_output(x)
    error = np.mean((t - prediction) ** 2)
    classifications.append((x, t, prediction, error))

print '--- Worst predictions: ---'
classifications = sorted(classifications, key = lambda (x, t, p, e): e, reverse=True)
print_classification(0)
print_classification(1)


print '--- Best predictions: ---'
print_classification(-1)
print_classification(-2)


### Determine baseline.

In [ ]:
## AVERAGE SQUARED MAGNITUDE OF SENSOR VALUES:

test_targets = train_data[1].get_value()
print 'Loss of mean-regressor: %f' % np.mean((test_targets - np.mean(test_targets)) ** 2)

### Plot training progress 

In [ ]:
progress_filenames = ['training-progresses/pickup_box_learning_curvetrain_costs.txt', 
                      'training-progresses/pickup_box_learning_curvevalidation_accuracies.txt']
